<a href="https://colab.research.google.com/github/narel14/AAI-501-Grp-7/blob/main/aai_501_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
print('hellow world')

hellow world


**Finish task**